# hvplotとPanelのインタラクティブ可視化チュートリアル

このノートブックでは、hvplotとPanelを使用したインタラクティブなデータ可視化を学びます。

## 目次
1. hvplotとPanelの基礎
2. 経済物理学への応用
3. ネットワーク科学への応用

## インストール

必要なライブラリをインストールします：

In [2]:
# 必要なライブラリのインストール
# !pip install hvplot panel pandas numpy networkx bokeh

In [3]:
import numpy as np
import pandas as pd
import hvplot.pandas
import panel as pn
import networkx as nx

# Panelの拡張機能を有効化
pn.extension()

## 1. hvplotとPanelの基礎

### hvplotの基本的な使い方

hvplotは、Pandasのデータフレームから簡単にインタラクティブなプロットを作成できます。

In [4]:
# サンプルデータの作成
dates = pd.date_range('2020-01-01', periods=365, freq='D')
df = pd.DataFrame({
    'date': dates,
    'value1': np.cumsum(np.random.randn(365)),
    'value2': np.cumsum(np.random.randn(365)),
    'value3': np.cumsum(np.random.randn(365))
})

# 基本的なプロット
df.hvplot.line(x='date', y=['value1', 'value2', 'value3'], 
               title='時系列データの例', 
               width=700, height=400)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

### Panelを使ったインタラクティブなダッシュボード

Panelを使用すると、ウィジェットとプロットを組み合わせてインタラクティブなダッシュボードを作成できます。

In [5]:
# インタラクティブなプロットの作成
def create_plot(n_points=100, plot_type='line'):
    x = np.linspace(0, 10, n_points)
    y = np.sin(x)
    df_temp = pd.DataFrame({'x': x, 'y': y})
    
    if plot_type == 'line':
        return df_temp.hvplot.line(x='x', y='y', title=f'サインカーブ ({n_points}点)')
    else:
        return df_temp.hvplot.scatter(x='x', y='y', title=f'サインカーブ ({n_points}点)')

# ウィジェットの作成
n_points_slider = pn.widgets.IntSlider(name='点の数', start=10, end=500, value=100)
plot_type_select = pn.widgets.Select(name='プロットタイプ', options=['line', 'scatter'])

# インタラクティブなダッシュボード
interactive_plot = pn.bind(create_plot, n_points=n_points_slider, plot_type=plot_type_select)

dashboard = pn.Column(
    '# インタラクティブなプロット',
    pn.Row(n_points_slider, plot_type_select),
    interactive_plot
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'0483150d-1d56-4bd6-95e1-227b4a32f364': {'version…

## 2. 経済物理学への応用

経済物理学では、金融市場のデータを物理学的な手法で分析します。
ここでは、株価の時系列データとリターン分布を可視化します。

In [6]:
# 模擬的な株価データの生成（幾何ブラウン運動）
def generate_stock_prices(S0=100, mu=0.0005, sigma=0.02, T=1, N=252):
    """
    幾何ブラウン運動で株価を生成
    S0: 初期価格
    mu: ドリフト項
    sigma: ボラティリティ
    T: 期間（年）
    N: ステップ数
    """
    dt = T / N
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(size=N)
    W = np.cumsum(W) * np.sqrt(dt)
    X = (mu - 0.5 * sigma**2) * t + sigma * W
    S = S0 * np.exp(X)
    return t, S

# 複数の株価データを生成
dates = pd.date_range('2023-01-01', periods=252, freq='D')
t1, stock1 = generate_stock_prices(S0=100, sigma=0.02)
t2, stock2 = generate_stock_prices(S0=150, sigma=0.03)
t3, stock3 = generate_stock_prices(S0=80, sigma=0.015)

stock_df = pd.DataFrame({
    'date': dates,
    '株式A': stock1,
    '株式B': stock2,
    '株式C': stock3
})

# 株価の可視化
price_plot = stock_df.hvplot.line(
    x='date', 
    y=['株式A', '株式B', '株式C'],
    title='株価の時系列',
    ylabel='価格',
    width=700,
    height=400
)

price_plot

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [7]:
# リターンの計算と分析
returns_df = stock_df.copy()
returns_df['リターンA'] = stock_df['株式A'].pct_change()
returns_df['リターンB'] = stock_df['株式B'].pct_change()
returns_df['リターンC'] = stock_df['株式C'].pct_change()
returns_df = returns_df.dropna()

# リターン分布のヒストグラム
hist_plot = returns_df.hvplot.hist(
    y=['リターンA', 'リターンB', 'リターンC'],
    bins=30,
    alpha=0.6,
    title='リターンの分布',
    width=700,
    height=400
)

hist_plot

:NdOverlay   [Variable]
   :Histogram   [リターンC]   (Count)

In [8]:
# インタラクティブな経済物理学ダッシュボード
def create_econophysics_dashboard(volatility=0.02, drift=0.0005):
    # 新しい株価データを生成
    t, prices = generate_stock_prices(S0=100, mu=drift, sigma=volatility, T=1, N=252)
    dates = pd.date_range('2023-01-01', periods=252, freq='D')
    df = pd.DataFrame({'date': dates, 'price': prices})
    df['return'] = df['price'].pct_change()
    df = df.dropna()
    
    # プロットの作成
    price_plot = df.hvplot.line(
        x='date', y='price', 
        title=f'株価 (σ={volatility}, μ={drift})',
        width=350, height=300
    )
    
    return_hist = df.hvplot.hist(
        y='return', bins=30,
        title='リターン分布',
        width=350, height=300
    )
    
    return pn.Row(price_plot, return_hist)

# パラメータのウィジェット
volatility_slider = pn.widgets.FloatSlider(
    name='ボラティリティ (σ)', 
    start=0.005, end=0.05, step=0.005, value=0.02
)
drift_slider = pn.widgets.FloatSlider(
    name='ドリフト (μ)', 
    start=-0.001, end=0.002, step=0.0001, value=0.0005
)

# インタラクティブなダッシュボード
interactive_econ = pn.bind(
    create_econophysics_dashboard, 
    volatility=volatility_slider, 
    drift=drift_slider
)

econ_dashboard = pn.Column(
    '## 経済物理学：株価シミュレーション',
    pn.Row(volatility_slider, drift_slider),
    interactive_econ
)

econ_dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'776050eb-fec5-45e5-bbcb-d8d51666498c': {'version…

### ボラティリティクラスタリング

金融市場では、大きな価格変動が連続して起こる傾向があります（ボラティリティクラスタリング）。

In [9]:
# ボラティリティの時系列可視化
returns_df['rolling_vol_A'] = returns_df['リターンA'].rolling(window=20).std()
returns_df['rolling_vol_B'] = returns_df['リターンB'].rolling(window=20).std()
returns_df['rolling_vol_C'] = returns_df['リターンC'].rolling(window=20).std()

vol_plot = returns_df.hvplot.line(
    x='date',
    y=['rolling_vol_A', 'rolling_vol_B', 'rolling_vol_C'],
    title='ローリングボラティリティ（20日）',
    ylabel='ボラティリティ',
    width=700,
    height=400
)

vol_plot

:NdOverlay   [Variable]
   :Curve   [date]   (value)

## 3. ネットワーク科学への応用

ネットワーク科学では、複雑なネットワークの構造を分析・可視化します。
ここでは、様々なネットワークモデルとその特性を探索します。

In [10]:
import hvplot.networkx as hvnx

# ランダムネットワークの生成
def create_network(network_type='random', n=30, param=0.2):
    """
    様々なタイプのネットワークを生成
    """
    if network_type == 'random':
        G = nx.erdos_renyi_graph(n, param)
    elif network_type == 'small_world':
        k = max(2, int(n * param))
        G = nx.watts_strogatz_graph(n, k, 0.3)
    elif network_type == 'scale_free':
        m = max(1, int(n * param))
        G = nx.barabasi_albert_graph(n, m)
    else:
        G = nx.complete_graph(n)
    
    return G

# ネットワークの可視化
G = create_network('scale_free', n=50, param=0.1)

# 次数中心性の計算
degree_centrality = nx.degree_centrality(G)
nx.set_node_attributes(G, degree_centrality, 'degree_centrality')

# ネットワークの可視化
network_plot = hvnx.draw(
    G,
    with_labels=False,
    node_color='degree_centrality',
    node_size=100,
    cmap='viridis',
    width=600,
    height=600,
    title='スケールフリーネットワーク'
)

network_plot

:Graph   [start,end]

In [11]:
# ネットワーク特性の分析
def analyze_network(G):
    """
    ネットワークの基本的な特性を計算
    """
    degrees = [G.degree(n) for n in G.nodes()]
    
    metrics = {
        'ノード数': G.number_of_nodes(),
        'エッジ数': G.number_of_edges(),
        '平均次数': np.mean(degrees),
        '密度': nx.density(G),
        'クラスタリング係数': nx.average_clustering(G) if G.number_of_edges() > 0 else 0
    }
    
    return metrics, degrees

metrics, degrees = analyze_network(G)

# 次数分布のプロット
degree_df = pd.DataFrame({'degree': degrees})
degree_hist = degree_df.hvplot.hist(
    y='degree',
    bins=20,
    title='次数分布',
    xlabel='次数',
    ylabel='頻度',
    width=500,
    height=400
)

print("ネットワーク特性:")
for key, value in metrics.items():
    print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

degree_hist

ネットワーク特性:
  ノード数: 50
  エッジ数: 225
  平均次数: 9.0000
  密度: 0.1837
  クラスタリング係数: 0.2600


:Histogram   [degree]   (Count)

In [12]:
# インタラクティブなネットワーク探索ダッシュボード
def create_network_dashboard(network_type='random', n_nodes=30, connectivity=0.2):
    # ネットワークの生成
    G = create_network(network_type, n_nodes, connectivity)
    
    # 中心性の計算
    degree_cent = nx.degree_centrality(G)
    nx.set_node_attributes(G, degree_cent, 'centrality')
    
    # ネットワークの可視化
    network_viz = hvnx.draw(
        G,
        with_labels=False,
        node_color='centrality',
        node_size=80,
        cmap='plasma',
        width=400,
        height=400,
        title=f'{network_type}ネットワーク'
    )
    
    # 特性の計算
    metrics, degrees = analyze_network(G)
    
    # 次数分布
    degree_df = pd.DataFrame({'degree': degrees})
    degree_hist = degree_df.hvplot.hist(
        y='degree',
        bins=min(15, max(degrees) + 1) if degrees else 10,
        title='次数分布',
        width=400,
        height=400
    )
    
    # メトリクスの表示
    metrics_text = "### ネットワーク特性\n\n"
    for key, value in metrics.items():
        if isinstance(value, float):
            metrics_text += f"- **{key}**: {value:.4f}\n"
        else:
            metrics_text += f"- **{key}**: {value}\n"
    
    return pn.Column(
        pn.Row(network_viz, degree_hist),
        pn.pane.Markdown(metrics_text)
    )

# ウィジェットの作成
network_type_select = pn.widgets.Select(
    name='ネットワークタイプ',
    options=['random', 'small_world', 'scale_free', 'complete']
)
n_nodes_slider = pn.widgets.IntSlider(
    name='ノード数',
    start=10, end=100, step=10, value=30
)
connectivity_slider = pn.widgets.FloatSlider(
    name='接続性パラメータ',
    start=0.05, end=0.5, step=0.05, value=0.2
)

# インタラクティブなダッシュボード
interactive_network = pn.bind(
    create_network_dashboard,
    network_type=network_type_select,
    n_nodes=n_nodes_slider,
    connectivity=connectivity_slider
)

network_dashboard = pn.Column(
    '## ネットワーク科学：インタラクティブ探索',
    pn.Row(network_type_select, n_nodes_slider, connectivity_slider),
    interactive_network
)

network_dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'926cfed8-ee69-4121-9dd9-c2fec99ccdb1': {'version…

### コミュニティ検出

ネットワーク内のコミュニティ（密に接続されたノードのグループ）を検出します。

In [13]:
from networkx.algorithms import community

# ネットワークの生成
G = nx.karate_club_graph()

# コミュニティ検出（Louvain法）
communities = community.greedy_modularity_communities(G)

# コミュニティの情報をノードに追加
community_map = {}
for i, comm in enumerate(communities):
    for node in comm:
        community_map[node] = i

nx.set_node_attributes(G, community_map, 'community')

# 可視化
community_plot = hvnx.draw(
    G,
    with_labels=True,
    node_color='community',
    node_size=200,
    cmap='Set3',
    width=700,
    height=700,
    title='空手クラブネットワークのコミュニティ検出'
)

print(f"検出されたコミュニティ数: {len(communities)}")
for i, comm in enumerate(communities):
    print(f"コミュニティ {i+1}: {len(comm)}ノード")

community_plot

検出されたコミュニティ数: 3
コミュニティ 1: 17ノード
コミュニティ 2: 9ノード
コミュニティ 3: 8ノード


:Overlay
   .Graph.I  :Graph   [start,end]   (weight)
   .Labels.I :Labels   [x,y]   (index)

## まとめ

このチュートリアルでは以下を学びました：

1. **hvplotとPanelの基礎**
   - インタラクティブなプロットの作成
   - ウィジェットを使ったダッシュボード構築

2. **経済物理学への応用**
   - 株価データの生成とシミュレーション
   - リターン分布の分析
   - ボラティリティクラスタリングの可視化

3. **ネットワーク科学への応用**
   - 様々なネットワークモデルの生成
   - ネットワーク特性の計算と可視化
   - コミュニティ検出

hvplotとPanelを組み合わせることで、データ駆動型の科学研究において強力なインタラクティブ可視化ツールを構築できます。